In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [28]:
data = pd.read_csv('Churn_Modelling.csv')
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [29]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [30]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [31]:
onehot_encoder_geo = OneHotEncoder()
print(data[['Geography']].shape)

print(data['Geography'].shape)
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

(10000, 1)
(10000,)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [32]:
# Get Feature names
onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_dataframe = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [33]:
data = pd.concat([data.drop(['Geography'], axis=1),geo_dataframe], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
# Save the encoder & Scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('label_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [38]:
X=data.drop('Exited', axis=1)
Y=data['Exited']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


Tensorflow Implementation

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [40]:
X_train.shape[1]

12

In [41]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  #input layer
    Dense(32, activation='relu'), #hidden layer
    Dense(1, activation='relu') #outputlayer
])

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the Model
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [62]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [63]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [64]:
# train the model
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3971 - accuracy: 0.8470 - val_loss: 0.4647 - val_accuracy: 0.8225
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4081 - accuracy: 0.8411 - val_loss: 0.5963 - val_accuracy: 0.8390
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3935 - accuracy: 0.8434 - val_loss: 0.3778 - val_accuracy: 0.8510
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3733 - accuracy: 0.8487 - val_loss: 0.4222 - val_accuracy: 0.8245
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3848 - accuracy: 0.8450 - val_loss: 0.3743 - val_accuracy: 0.8455
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3558 - accuracy: 0.8576 - val_loss: 0.3547 - val_accuracy: 0.8630
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3920 - accuracy: 0.8438 - val_loss: 0.3889 - val_accuracy: 0.8435

In [65]:
model.save('model.h5')

c:\Users\prems\Documents\Work\genai\ANN\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [66]:
# Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6006 (pid 28556), started 0:03:04 ago. (Use '!kill 28556' to kill it.)

: 

In [ ]:
#  Load Pickle file
